# Model-Assisted Labeling for *video* in Labelbox

## Imports

In [ ]:
!pip install labelbox[data]

In [3]:
import labelbox.data
import ndjson
import requests
import json
import uuid
import requests
import numpy as np
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client
# from image_model import predict, class_mappings, load_model
# from image_mal_utils import (visualize_bbox_ndjsons, visualize_poly_ndjsons,
                            #  visualize_point_ndjsons, visualize_mask_ndjsons)
# from io import BytesIO
# from getpass import getpass
# from PIL import Image
# from google.cloud import storage
# import datetime as dt
# from skimage import measure
# import os

# from labelbox.data.annotation_types import (
#     Label,
#     LabelList,
#     VideoData,
#     Rectangle,
#     Point,
#     VideoObjectAnnotation
# )

## Connecting to project

In [4]:
LB_API_KEY = ''  # Model-assisted_Labeling key
# Create Labelbox client
client = Client(api_key=LB_API_KEY)
#Test Project: testing_mal_project
project = client.get_project('ckty9de742d3e0y8o8e617n5c')
ontology = project.ontology()
#test_copy_100testimages_s25p_f10.mp4 from "Test Dataset_Lions&Ants" Dataset
datarow_id = 'cktkcb3zhnh5l0yrf6d9oezre'

In [5]:
project.enable_model_assisted_labeling()

True

Conecting to the ontology, so we can use schemas to upload annotations.

In [6]:
class2id = {"ant": 0,
            "ant-head": 1,
            "trophallaxis-ant": 2,
            "larva": 3,
            "trophallaxis-larva": 4,
            "food-noise": 5,
            "pupa": 6,
            "barcode": 7,
            "uncategorized": 8}

ontology_builder = OntologyBuilder(tools=[Tool(tool=Tool.Type.BBOX, name=key) for key in class2id])
# So lets reconstruct the ontology builder with all of the ids.
ontology = ontology_builder.from_project(project)
# We want all of the feature schemas to be easily accessible by name.
schema_lookup = {class2id[tool.name]: tool.feature_schema_id for tool in ontology.tools}
print(schema_lookup)

{0: 'ckty9dfw44f8h0y9w0cnje3yr', 1: 'ckty9dfw44f8j0y9w9jgo7zx4', 2: 'ckty9dfw54f8l0y9wb6ig7vu4', 3: 'ckty9dfw54f8n0y9wcrb65ies', 4: 'ckty9dfw54f8p0y9w4qxygs9m', 5: 'ckty9dfw64f8r0y9wcu08h7ca', 6: 'ckty9dfw64f8t0y9wb3lv6svx', 7: 'ckty9dfw64f8v0y9wfnwl2fht', 8: 'ckty9dfw64f8x0y9w4kske8vf'}


In [7]:
def create_boxes_ndjson(filepath: str, datarow_id: str, framenum: int = 1) -> list:
    """
    Creates annotations using txt file with model predictions according to Lablebox style.
    
    Args:
        filepath (str): path to an txt annotation file of the YOLO format
        datarow_id (str): id of the data_row to add this annotation to
        framenum (int): the number of the frame, which was annotated in a current file
        
    Returns:
        json representation of a bounding box
    """
    annotations = []
    with open(filepath, 'r') as f:
      for line in f:
        cls, center_x, center_y, w, h = tuple(map(lambda x: float(x), line.split()))
        annotations.append({"uuid": str(uuid.uuid4()),
                            "schemaId": schema_lookup[int(cls)],
                            "dataRow": {
                              "id": datarow_id
                            },
                            "segments": [
                              {
                                "keyframes": [
                                  {
                                    "frame": framenum,
                                    "bbox": {
                                      "top": center_y - h / 2,
                                      "left": center_x - w / 2,
                                      "height": h,
                                      "width": w
                                      }
                                  }]
                              }]
        })

    return annotations

## Upploading annotations *to* Lablebox.

In [51]:
upload_task = project.upload_annotations(name=f"upload-job-{uuid.uuid4()}",
                                         annotations=annotations,
                                         validate=False)
upload_task.wait_until_done()
# Review the upload status
print(upload_task.errors) 

[]


## Downloading json *from* Lablebox.

Downloading json from Lablebox. Be attentive, you can do it either using labeler email or External ID (video name). And if your uploaded annotations were not changed by a laberer on the labelbox website then they can not be available to download.

In [13]:
export_url = project.export_labels()
exports = requests.get(export_url).json()
# exports
for el in exports:
  if el['External ID'] and 'test_copy_100testimages_s25p_f10' in el['External ID']:
    print(el)
    annotations_url = el["Label"]["frames"]

In [31]:
# annotations_url = 'https://api.labelbox.com/v1/frames/cky1g4oze0z1y0zapb0b76rcs'
headers = {"Authorization": f"Bearer {LB_API_KEY}"}
annotations = ndjson.loads(requests.get(annotations_url, headers=headers).text)

# fname = ''.join(('_'.join(labels[i]['Dataset Name'].split()), '-', lbemail.split('@')[0], '.json'))
with open('file.json', 'w') as fout:
	json.dump(annotations, fout)

In [9]:
vlabels = project.video_label_generator()
# Export labels as a json:
labels = project.export_labels(download = True)

In [11]:
create_boxes_ndjson('1.txt', 'cktkcb3zhnh5l0yrf6d9oezre', 1)

[{'dataRow': {'id': 'cktkcb3zhnh5l0yrf6d9oezre'},
  'schemaId': 'ckty9dfw44f8h0y9w0cnje3yr',
  'segments': [{'keyframes': [{'bbox': {'height': 388.0,
       'left': 34.0,
       'top': 373.0,
       'width': 400.0},
      'frame': 1}]}],
  'uuid': 'e38178d8-22bf-44a6-8d1c-1f85d864b761'},
 {'dataRow': {'id': 'cktkcb3zhnh5l0yrf6d9oezre'},
  'schemaId': 'ckty9dfw44f8j0y9w9jgo7zx4',
  'segments': [{'keyframes': [{'bbox': {'height': 100.0,
       'left': 184.0,
       'top': 517.0,
       'width': 100.0},
      'frame': 1}]}],
  'uuid': 'b7c1ae1b-f32d-425c-9793-b8e77aebc46f'}]